<a href="https://colab.research.google.com/github/Junior11995/Challenge-Telecom-X-an-lisis-de-evasi-n-de-clientes/blob/main/desaf_o_telecom_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Encabezado y librerías




In [1]:
# --- Encabezado ---
# =========================================================
# 0) ENCABEZADO / CONFIG
# =========================================================
PROYECTO = "Telecom X - Churn"
AUTOR = "Junior Valera"
REPO = "https://github.com/Junior11995/Challenge-Telecom-X-an-lisis-de-evasi-n-de-clientes"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from pandas import json_normalize

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

def normalize_columns(cols: pd.Index) -> pd.Index:
    return (cols
            .str.strip().str.lower()
            .str.replace(r"\s+", "_", regex=True)
            .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))

def quality_report(df: pd.DataFrame) -> pd.DataFrame:
    rep = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "n_null": df.isna().sum(),
        "pct_null": df.isna().mean().round(4),
        "n_unique": df.nunique(dropna=True)
    })
    rep["example_values"] = [df[c].dropna().unique()[:5] for c in df.columns]
    return rep

print(PROYECTO, AUTOR, REPO)



Telecom X - Churn Junior Valera https://github.com/Junior11995/Challenge-Telecom-X-an-lisis-de-evasi-n-de-clientes


# Extracción de datos desde API JSON (TelecomX_Data.json)

In [2]:
# === EXTRACCIÓN DESDE LA API (JSON) ===
# Fuente oficial (ramo main del repo que compartiste):
URL_FUENTE = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

import pandas as pd
import requests
from pandas import json_normalize

# Descarga segura
resp = requests.get(URL_FUENTE, timeout=30)
resp.raise_for_status()
data = resp.json()

# Conversión robusta a DataFrame
if isinstance(data, list):
    df_raw = pd.DataFrame(data)
elif isinstance(data, dict):
    try:
        df_raw = json_normalize(data, max_level=1)
    except Exception:
        df_raw = pd.DataFrame([data])
else:
    raise ValueError("Formato JSON no soportado.")

print("RAW shape:", df_raw.shape)
display(df_raw.head(5))

# (Opcional) guardar copia cruda para trazabilidad
df_raw.to_csv("telecomx_raw.csv", index=False)

RAW shape: (7267, 6)


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


# Conoce el conjunto de datos

In [4]:
# =========================================================
# CONOCE EL CONJUNTO DE DATOS (perfilado inicial)
# Requiere: df_raw ya cargado desde la etapa de Extracción
# =========================================================

import pandas as pd
import numpy as np

# Trabajamos sobre una copia normalizada
df = df_raw.copy()
df.columns = (df.columns
              .str.strip().str.lower()
              .str.replace(r"\s+", "_", regex=True)
              .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))

# 1) Vista general
print("Shape (filas, columnas):", df.shape)
display(df.head(3))
df.info()          # tipos y nulos a alto nivel
display(df.dtypes) # tipos por columna

# 2) Resumen de calidad por columna
perfil = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_null": df.isna().sum(),
    "pct_null": df.isna().mean().round(4),
    "n_unique": df.nunique(dropna=True)
}).sort_values(["pct_null","n_unique"], ascending=[False, True])
display(perfil.head(20))

# 3) Posibles columnas objetivo (churn/baja/cancelación)
target_candidates = [c for c in df.columns
                     if any(k in c for k in ["churn","baja","cancel","evas"])]

print("Posibles columnas objetivo:", target_candidates)

# Elegimos provisionalmente la 1ª columna candidata que parezca binaria
target = None
for c in target_candidates:
    if df[c].astype(str).str.lower().isin(["0","1","true","false","yes","no","si","sí"]).any():
        target = c
        break
if target is None and len(target_candidates) > 0:
    target = target_candidates[0]
print("Target provisional:", target)

# 4) Separación de numéricas y categóricas para orientar el EDA
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in df.columns if c not in num_cols]

print(f"Número de columnas numéricas: {len(num_cols)}")
print(f"Número de columnas categóricas: {len(cat_cols)}")

# 5) Top columnas potencialmente informativas:
#    - numéricas con baja proporción de nulos y varianza > 0
#    - categóricas con cardinalidad moderada (2..30 categorías)
informativas_num = [c for c in num_cols
                    if df[c].notna().mean() > 0.9 and df[c].nunique() > 1]
informativas_cat = [c for c in cat_cols
                    if 2 <= df[c].nunique(dropna=True) <= 30]

print("Numéricas informativas (candidatas):", informativas_num[:10])
print("Categóricas informativas (candidatas):", informativas_cat[:10])

# 6) Si hay target binaria, medimos relación rápida:
if target is not None:
    # Codificamos target a 0/1 de forma robusta
    y = df[target].astype(str).str.lower().map(
        {"1":1,"true":1,"yes":1,"si":1,"sí":1,
         "0":0,"false":0,"no":0}
    )
    # Si sigue con NaN (p.ej. 'No'/'Yes' capitalizado o 'Y'/'N'), intentamos fallback
    if y.isna().mean() > 0.3:
        y = (df[target].astype(str).str.lower().isin(["1","true","yes","si","sí","y"])).astype(int)

    # a) Correlación punto-biserial aproximada para numéricas
    import math
    corr_num = {}
    for c in informativas_num:
        s = pd.to_numeric(df[c], errors="coerce")
        m = pd.concat([s, y], axis=1).dropna()
        if m[target].nunique() == 2 and m[c].nunique() > 1:
            try:
                corr_num[c] = m[c].corr(m[target])
            except Exception:
                pass
    corr_num = pd.Series(corr_num).sort_values(key=lambda s: s.abs(), ascending=False).head(10)
    print("\nTop numéricas por |correlación| con target:")
    display(corr_num)

    # b) Dispersión de tasas de churn por categoría (gap máx - min) como señal
    churn_spread = {}
    for c in informativas_cat[:20]:
        tmp = pd.concat([df[c], y], axis=1).dropna()
        if tmp[c].nunique() >= 2:
            rates = tmp.groupby(c)[target].mean()
            churn_spread[c] = float(rates.max() - rates.min())
    churn_spread = pd.Series(churn_spread).sort_values(ascending=False).head(10)
    print("\nTop categóricas por diferencia de tasas de churn (max-min):")
    display(churn_spread)

Shape (filas, columnas): (7267, 6)


,customerid,churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerid  7267 non-null   object
 1   churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


,0
customerid,object
churn,object
customer,object
phone,object
internet,object
account,object


TypeError: unhashable type: 'dict'

In [5]:
# =========================================================
# REPARACIÓN: Aplanar columnas anidadas + perfilado seguro
# Requiere: df_raw ya cargado (con columnas: customer, phone, internet, account)
# =========================================================
import json
from pandas import json_normalize

# 1) Aplanado de columnas anidadas
cols_nested = [c for c in ["customer", "phone", "internet", "account"] if c in df_raw.columns]
parts = []
for c in cols_nested:
    # json_normalize sobre cada columna de dicts
    block = json_normalize(df_raw[c])
    block = block.add_prefix(f"{c}_")
    parts.append(block)

df_flat = pd.concat([df_raw.drop(columns=cols_nested, errors="ignore")] + parts, axis=1)

# 2) Normalización de nombres
def _normalize_cols(cols):
    return (cols
            .str.strip().str.lower()
            .str.replace(r"\s+", "_", regex=True)
            .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8"))

df = df_flat.copy()
df.columns = _normalize_cols(df.columns)

# 3) Opcional: limpieza rápida de churn a texto estándar
if "churn" in df.columns:
    df["churn"] = (df["churn"].astype(str)
                   .str.strip().str.lower())

# 4) Perfilado seguro (evita error de unhashable dict)
def _make_hashable(x):
    # Serializa estructuras no hasheables para poder contar valores únicos
    if isinstance(x, dict):
        return json.dumps(x, sort_keys=True)
    if isinstance(x, (list, tuple)):
        return json.dumps(x, sort_keys=True)
    if isinstance(x, set):
        return json.dumps(sorted(list(x)))
    return x

def safe_nunique(series: pd.Series) -> int:
    return series.map(_make_hashable).nunique(dropna=True)

perfil = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_null": df.isna().sum(),
    "pct_null": df.isna().mean().round(4),
    "n_unique": [safe_nunique(df[c]) for c in df.columns]
}).sort_values(["pct_null", "n_unique"], ascending=[False, True])

print("Shape aplanado:", df.shape)
display(df.head(3))
df.info()
display(perfil.head(20))

# 5) Guardar copia intermedia (opcional, útil para trazabilidad)
df.to_csv("telecomx_flat.csv", index=False)
print("Guardado intermedio: telecomx_flat.csv")


Shape aplanado: (7267, 21)


,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,internet_onlinesecurity,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges.monthly,account_charges.total
0,0002-ORFBO,no,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,no,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7267 non-null   object 
 1   churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_seniorcitizen     7267 non-null   int64  
 4   customer_partner           7267 non-null   object 
 5   customer_dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_phoneservice         7267 non-null   object 
 8   phone_multiplelines        7267 non-null   object 
 9   internet_internetservice   7267 non-null   object 
 10  internet_onlinesecurity    7267 non-null   object 
 11  internet_onlinebackup      7267 non-null   object 
 12  internet_deviceprotection  7267 non-null   object 
 13  internet_techsupport       7267 non-null   objec

,dtype,n_null,pct_null,n_unique
customer_gender,object,0,0.00,2
customer_seniorcitizen,int64,0,0.00,2
customer_partner,object,0,0.00,2
customer_dependents,object,0,0.00,2
phone_phoneservice,object,0,0.00,2
account_paperlessbilling,object,0,0.00,2
churn,object,0,0.00,3
phone_multiplelines,object,0,0.00,3
internet_internetservice,object,0,0.00,3
internet_onlinesecurity,object,0,0.00,3


Guardado intermedio: telecomx_flat.csv


## Casting de tipos

In [6]:
# =========================================================
# TRANSFORM: CASTING DE TIPOS + DUPLICADOS + NULOS
# df ya aplanado y normalizado (del bloque anterior)
# Salida: df_cast con tipos corregidos
# =========================================================

import pandas as pd
import numpy as np

df_cast = df.copy()

# --- 1) Estandarización de textos "Yes/No/Si/No" -> booleanos (True/False) ---
def to_bool(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()
    true_set  = {"yes", "si", "sí", "true", "1"}
    false_set = {"no", "false", "0"}
    out = pd.Series(index=series.index, dtype="float")  # temporal
    out[s.isin(true_set)] = 1
    out[s.isin(false_set)] = 0
    return out.astype("float")

bool_like_cols = [c for c in df_cast.columns if any(k in c for k in [
    "phoneservice","multiplelines",
    "onlinesecurity","onlinebackup","deviceprotection",
    "techsupport","streamingtv","streamingmovies",
    "paperlessbilling"
])]
for c in bool_like_cols:
    try:
        df_cast[c] = to_bool(df_cast[c])
    except Exception:
        pass

# --- 2) Casting numérico para cargos/tenure/seniorcitizen ---
num_candidates = [c for c in df_cast.columns if any(k in c for k in [
    "charges","tenure","seniorcitizen"
])]
for c in num_candidates:
    df_cast[c] = pd.to_numeric(df_cast[c], errors="coerce")

# --- 3) Casting de fechas (heurístico por nombre) ---
date_candidates = [c for c in df_cast.columns if any(k in c for k in [
    "fecha","date","start","end"
])]
for c in date_candidates:
    df_cast[c] = pd.to_datetime(df_cast[c], errors="coerce")

# --- 4) Normalizar target churn a 0/1 si existe ---
if "churn" in df_cast.columns:
    ytmp = df_cast["churn"].astype(str).str.strip().str.lower()
    df_cast["churn"] = np.where(ytmp.isin(["yes","si","sí","true","1"]), 1,
                         np.where(ytmp.isin(["no","false","0"]), 0, np.nan)).astype("float")

# --- 5) Duplicados (por customerid si existe, si no por fila completa) ---
if "customerid" in df_cast.columns:
    dups = df_cast.duplicated(subset=["customerid"]).sum()
    df_cast = df_cast.drop_duplicates(subset=["customerid"])
else:
    dups = df_cast.duplicated().sum()
    df_cast = df_cast.drop_duplicates()

print(f"Duplicados eliminados: {dups}")

# --- 6) Reporte de nulos y tipos tras casting ---
report = pd.DataFrame({
    "dtype": df_cast.dtypes.astype(str),
    "n_null": df_cast.isna().sum(),
    "pct_null": df_cast.isna().mean().round(4),
    "n_unique": [df_cast[c].nunique(dropna=True) for c in df_cast.columns]
}).sort_values(["pct_null","n_unique"], ascending=[False, True])

print("Shape post-casting:", df_cast.shape)
display(report.head(20))

Duplicados eliminados: 0
Shape post-casting: (7267, 21)


/tmp/ipython-input-3085228093.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cast[c] = pd.to_datetime(df_cast[c], errors="coerce")
/tmp/ipython-input-3085228093.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cast[c] = pd.to_datetime(df_cast[c], errors="coerce")


,dtype,n_null,pct_null,n_unique
customer_gender,datetime64[ns],7267,1.00,0
customer_dependents,datetime64[ns],7267,1.00,0
internet_onlinesecurity,float64,1581,0.22,2
internet_onlinebackup,float64,1581,0.22,2
internet_deviceprotection,float64,1581,0.22,2
internet_techsupport,float64,1581,0.22,2
internet_streamingtv,float64,1581,0.22,2
internet_streamingmovies,float64,1581,0.22,2
phone_multiplelines,float64,707,0.10,2
churn,float64,224,0.03,2


In [7]:
# =========================================================
# IMPUTACIÓN CONTROLADA
#   - Actívala poniendo DO_IMPUTE = True
#   - Imputa medianas en numéricas y moda en categóricas
# =========================================================
DO_IMPUTE = True  # <-- cambia a True si quieres imputar ahora

df_ready = df_cast.copy()
if DO_IMPUTE:
    num_cols = [c for c in df_ready.columns if pd.api.types.is_numeric_dtype(df_ready[c])]
    cat_cols = [c for c in df_ready.columns if c not in num_cols]

    for c in num_cols:
        if df_ready[c].isna().any():
            df_ready[c] = df_ready[c].fillna(df_ready[c].median())

    for c in cat_cols:
        if df_ready[c].isna().any():
            moda = df_ready[c].mode(dropna=True)
            if len(moda) > 0:
                df_ready[c] = df_ready[c].fillna(moda.iloc[0])

    print("Imputación aplicada.")
else:
    print("Imputación desactivada. (Configura DO_IMPUTE=True para aplicarla)")

# Guarda snapshot de transformación (útil para trazabilidad)
df_ready.to_csv("telecomx_ready_transform.csv", index=False)
print("Guardado: telecomx_ready_transform.csv")

Imputación aplicada.
Guardado: telecomx_ready_transform.csv


# COMPROBACIÓN DE INCOHERENCIAS EN LOS DATOS

In [8]:
# =========================================================
# COMPROBACIÓN DE INCOHERENCIAS EN LOS DATOS
# Requiere: df_ready (o df_cast / df) ya aplanado/transformado
# Objetivo: detectar nulos/duplicados residuales, errores de formato
#           e inconsistencias en categorías/fechas/números.
# Tips usados: pandas.unique(), Series.dt.normalize()
# =========================================================
import pandas as pd
import numpy as np

# 0) Selección robusta del DataFrame de trabajo
source_df = globals().get("df_ready", globals().get("df_cast", globals().get("df", None)))
assert source_df is not None, "No se encontró df_ready/df_cast/df. Ejecuta los bloques previos."
dfchk = source_df.copy()

results = {}

# 1) Nulos y duplicados residuales
results["nulls_by_col"] = (
    pd.DataFrame({
        "n_null": dfchk.isna().sum(),
        "pct_null": dfchk.isna().mean().round(4)
    })
    .sort_values(["pct_null","n_null"], ascending=[False, False])
)

if "customerid" in dfchk.columns:
    results["dups_on_customerid"] = int(dfchk.duplicated(subset=["customerid"]).sum())
    results["dups_rows_total"]    = int(dfchk.duplicated().sum())
else:
    results["dups_on_customerid"] = None
    results["dups_rows_total"]    = int(dfchk.duplicated().sum())

# 2) Inconsistencias en categorías (espacios/case/variantes de Yes/No)
cat_cols = [c for c in dfchk.columns if dfchk[c].dtype == "object"]
def norm_txt(s: pd.Series) -> pd.Series:
    return (s.astype(str).str.strip()
            .str.replace(r"\s+", " ", regex=True)
            .str.lower())

cat_issues = []
for c in cat_cols:
    raw_uniques = pd.unique(dfchk[c].dropna())  # pandas.unique()
    if len(raw_uniques) <= 50:  # solo columnas de cardinalidad moderada
        s_norm = norm_txt(dfchk[c].dropna())
        norm_uniques = pd.unique(s_norm)
        # ¿Hay diferencias entre valores crudos y normalizados?
        has_space_or_case = any(a != b for a, b in zip(sorted(map(str, raw_uniques)), sorted(map(str, norm_uniques))) ) \
                            or (len(raw_uniques) != len(norm_uniques))
        # Detección simple de booleans tipo Yes/No
        yes_set  = {"yes", "si", "sí", "true", "1"}
        no_set   = {"no", "false", "0"}
        binary_like = set(norm_uniques).issubset(yes_set.union(no_set)) and len(norm_uniques) <= 5
        cat_issues.append({
            "column": c,
            "n_unique_raw": int(len(raw_uniques)),
            "n_unique_norm": int(len(norm_uniques)),
            "has_space_or_case_inconsistency": bool(has_space_or_case),
            "looks_binary_yes_no": bool(binary_like),
            "sample_raw": list(map(str, raw_uniques[:8])),
            "sample_norm": list(map(str, norm_uniques[:8]))
        })

results["categorical_inconsistencies"] = pd.DataFrame(cat_issues).sort_values(
    ["has_space_or_case_inconsistency","n_unique_norm"], ascending=[False, True]
)

# 3) Números: negativos/imposibles y outliers por z-score
num_cols = [c for c in dfchk.columns if pd.api.types.is_numeric_dtype(dfchk[c])]
num_issues = []
for c in num_cols:
    s = pd.to_numeric(dfchk[c], errors="coerce")
    neg = int((s < 0).sum())
    # Reglas de negocio heurísticas
    maybe_non_negative = any(k in c for k in ["charge","tenure","seniorcitizen","months","total"])
    invalid_binary = None
    if "seniorcitizen" in c:
        invalid_binary = int(~s.isin([0,1]).sum()) if s.notna().any() else None
    # Outliers (|z|>4) si hay varianza
    z_out = None
    if s.notna().sum() > 10 and s.std(ddof=0) > 0:
        z = (s - s.mean())/s.std(ddof=0)
        z_out = int((z.abs() > 4).sum())
    num_issues.append({
        "column": c,
        "n_negative": neg if maybe_non_negative else None,
        "invalid_binary_seniorcitizen": invalid_binary,
        "z_outliers_gt4": z_out
    })

results["numeric_inconsistencies"] = pd.DataFrame(num_issues).sort_values(
    ["n_negative","z_outliers_gt4"], ascending=[False, False]
)

# 4) Fechas: normalización y orden lógico (start <= end)
date_cols = [c for c in dfchk.columns if pd.api.types.is_datetime64_any_dtype(dfchk[c])]
date_report = []
for c in date_cols:
    s = dfchk[c]
    # ¿Hay componente de tiempo? Comparamos con s.dt.normalize()
    try:
        normalized = s.dt.normalize()  # Series.dt.normalize()
        has_time_component = bool((s.dropna() != normalized.dropna()).any())
    except Exception:
        has_time_component = None
    date_report.append({
        "column": c,
        "has_time_component": has_time_component,
        "min": s.min(),
        "max": s.max(),
        "n_null": int(s.isna().sum())
    })

# Heurística: pares start/end por nombre
def find_pair(cols, key_start="start", key_end="end"):
    s = [c for c in cols if key_start in c]
    e = [c for c in cols if key_end in c]
    return s, e

starts, ends = find_pair(date_cols, "start", "end")
pair_issues = []
for scol in starts:
    for ecol in ends:
        comp = dfchk[[scol, ecol]].dropna()
        if not comp.empty:
            inversions = int((comp[scol] > comp[ecol]).sum())
            pair_issues.append({"start_col": scol, "end_col": ecol, "n_start_gt_end": inversions})

results["date_inconsistencies"] = pd.DataFrame(date_report)
results["date_pair_order_issues"] = pd.DataFrame(pair_issues)

# 5) Identificadores: formato de customerid
if "customerid" in dfchk.columns:
    s = dfchk["customerid"].astype(str)
    # patrón típico del dataset: "0002-ORFBO" (4 dígitos + "-" + 5 letras)
    pattern = r"^\d{4}-[A-Z]{5}$"
    bad_fmt = int(~s.str.match(pattern, na=False).sum())
    results["customerid_format_bad"] = bad_fmt
else:
    results["customerid_format_bad"] = None

# 6) Salidas visibles
print("=== NULOS POR COLUMNA ===")
display(results["nulls_by_col"].head(25))

print("\n=== DUPLICADOS ===")
print("Duplicados por customerid:", results["dups_on_customerid"])
print("Duplicados por fila completa:", results["dups_rows_total"])

print("\n=== CATEGORÍAS CON POSIBLES INCONSISTENCIAS ===")
display(results["categorical_inconsistencies"].head(20))

print("\n=== NUMÉRICAS CON POSIBLES INCONSISTENCIAS ===")
display(results["numeric_inconsistencies"].head(20))

print("\n=== FECHAS (NORMALIZACIÓN Y RANGO) ===")
display(results["date_inconsistencies"])
print("\n=== PARES start/end CON start > end ===")
display(results["date_pair_order_issues"])

print("\n=== CUSTOMERID FORMATO INVÁLIDO (aprox.) ===")
print(results["customerid_format_bad"])

=== NULOS POR COLUMNA ===


,n_null,pct_null
customer_gender,7267,1.00
customer_dependents,7267,1.00
customerid,0,0.00
churn,0,0.00
customer_seniorcitizen,0,0.00
customer_partner,0,0.00
customer_tenure,0,0.00
phone_phoneservice,0,0.00
phone_multiplelines,0,0.00
internet_internetservice,0,0.00



=== DUPLICADOS ===
Duplicados por customerid: 0
Duplicados por fila completa: 0

=== CATEGORÍAS CON POSIBLES INCONSISTENCIAS ===


,column,n_unique_raw,n_unique_norm,has_space_or_case_inconsistency,looks_binary_yes_no,sample_raw,sample_norm
0,customer_partner,2,2,True,True,"[Yes, No]","[yes, no]"
1,internet_internetservice,3,3,True,False,"[DSL, Fiber optic, No]","[dsl, fiber optic, no]"
2,account_contract,3,3,True,False,"[One year, Month-to-month, Two year]","[one year, month-to-month, two year]"
3,account_paymentmethod,4,4,True,False,"[Mailed check, Electronic check, Credit card (...","[mailed check, electronic check, credit card (..."



=== NUMÉRICAS CON POSIBLES INCONSISTENCIAS ===


...


=== FECHAS (NORMALIZACIÓN Y RANGO) ===


,column,has_time_component,min,max,n_null
0,customer_gender,False,NaT,NaT,7267
1,customer_dependents,False,NaT,NaT,7267



=== PARES start/end CON start > end ===


""



=== CUSTOMERID FORMATO INVÁLIDO (aprox.) ===
-7268
